In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/pytorch/vision.git
import sys
sys.path.append('/content/vision/')
import locale
locale.getpreferredencoding = lambda: "UTF-8"

Cloning into 'vision'...
remote: Enumerating objects: 370557, done.
remote: Counting objects: 100% (20797/20797), done.
remote: Compressing objects: 100% (1332/1332), done.
remote: Total 370557 (delta 19527), reused 20577 (delta 19375), pack-reused 349760
Receiving objects: 100% (370557/370557), 740.85 MiB | 47.89 MiB/s, done.
Resolving deltas: 100% (342056/342056), done.


In [ ]:
from torchvision.models.segmentation.deeplabv3 import DeepLabV3_ResNet101_Weights
from torchvision.models.segmentation.deeplabv3 import DeepLabHead
import torchvision.models.segmentation as segmentation
import torch.nn as nn

weights = DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1
model = segmentation.deeplabv3_resnet101(pretrained=False, progress=True, num_classes=21, aux_loss=None)
state_dict = weights.get_state_dict(progress=True)
in_channels = model.classifier[4].in_channels
model.classifier[4] = nn.Conv2d(in_channels, 26, kernel_size=1)
num_classes = model.classifier[4].out_channels

# print(model)
print(num_classes)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 258MB/s]
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet101_coco-586e9e4e.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet101_coco-586e9e4e.pth
100%|██████████| 233M/233M [00:00<00:00, 261MB/s]


26


In [ ]:
from PIL import Image
from torchvision import transforms
import random
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from PIL import Image

transform = transforms.Compose([
    transforms.Pad((0, 396)),
    transforms.Resize((513, 513)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.45969197, 0.44145255, 0.45046159],
                         std=[0.27247444, 0.27013482, 0.27866579])
])

dataset = datasets.ImageFolder(root="/content/drive/MyDrive/2DSSs", transform=transform)
indices = random.sample(range(len(dataset)), 4)
subset_dataset = Subset(dataset, indices)
dataloader = DataLoader(subset_dataset, batch_size=4)
images, labels = next(iter(dataloader))
print(f'images shape: {images.shape}')

model.train()
output_train = model(images)
print(type(output_train))
output_shape = output_train['out'].shape
print(f'train mode output shape: {output_shape}')

model.eval()
output_eval = model(images)
print(type(output_eval))
output_shape = output_eval['out'].shape
print(f'eval mode output shape: {output_shape}')

images shape: torch.Size([4, 3, 513, 513])
<class 'collections.OrderedDict'>
train mode output shape: torch.Size([4, 26, 513, 513])
<class 'collections.OrderedDict'>
eval mode output shape: torch.Size([4, 26, 513, 513])
